In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [2]:
xqrs = pd.read_csv('../to_csv/xqrs_v2.csv').drop('Unnamed: 0', axis=1)
xqrs.head(10)

,Set,Categorie Montage,Dossier Patient,Patient,Session,File N°,exam_duration,SWT_vs_XQRS,Pan_vs_XQRS,interval_index,interval_start_time,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
0,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,0.0,0.0,NaN,NaN,NaN,5.0,26.315789,6.0,31.578947,NaN,NaN,179.687500,NaN,NaN,NaN,132.987013,95.108359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,1.0,10000.0,537.109375,56.359305,41.722014,4.0,21.052632,4.0,21.052632,41.722578,546.875000,189.453125,0.077680,0.104931,113.041243,141.566820,97.834395,13.134853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,2.0,20000.0,602.213542,51.251884,26.831113,4.0,22.222222,4.0,22.222222,27.702479,585.937500,125.000000,0.046001,0.085106,100.296032,108.936170,88.786127,8.031925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,3.0,30000.0,564.453125,33.125314,21.511084,1.0,5.555556,2.0,11.111111,21.687087,548.828125,89.843750,0.038421,0.058686,106.617186,110.503597,94.814815,5.605881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,4.0,40000.0,NaN,NaN,NaN,2.0,11.111111,2.0,11.111111,NaN,NaN,125.000000,NaN,NaN,NaN,139.636364,108.169014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,5.0,50000.0,NaN,NaN,NaN,4.0,20.000000,6.0,30.000000,NaN,NaN,209.960938,NaN,NaN,NaN,129.894292,89.302326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,6.0,60000.0,NaN,NaN,NaN,0.0,0.000000,1.0,5.263158,NaN,NaN,31.250000,NaN,NaN,NaN,109.714286,103.783784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,7.0,70000.0,NaN,NaN,NaN,3.0,16.666667,4.0,22.222222,NaN,NaN,160.156250,NaN,NaN,NaN,115.488722,88.275862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,NaN,0.0
8,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,8.0,80000.0,580.512153,48.915618,20.916869,1.0,5.555556,2.0,11.111111,22.409678,562.500000,136.718750,0.038603,0.084263,103.974882,109.714286,87.771429,7.551172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,NaN,0.0
9,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,9.0,90000.0,NaN,NaN,NaN,3.0,16.666667,4.0,22.222222,NaN,NaN,132.812500,NaN,NaN,NaN,137.142857,105.205479,NaN,NaN,NaN,NaN,NaN,3.376083,4.288617,864.824552,0.255768,18.968881,64.040522,3.376083,0.0


In [3]:
xqrs['Dossier Patient'] = xqrs['Dossier Patient'].astype(str)
xqrs['Patient'] = xqrs['Patient'].astype(str)

In [4]:
xqrs['Key'] = xqrs.Set + '_' + xqrs['Categorie Montage'] + '_' + xqrs['Patient'] + '_' + xqrs.Session + '_' + xqrs['File N°']
len(xqrs.Key.unique())

5080

In [5]:
xqrs.drop(['Set', 'Categorie Montage', 'exam_duration', 'interval_start_time', 'interval_index', 'Session', 'File N°', 'Dossier Patient'], axis=1, inplace=True)

In [6]:
xqrs = xqrs[(xqrs['Pan_vs_XQRS'] >= 0.9) | (xqrs['SWT_vs_XQRS'] >= 0.9)]
xqrs.shape

(238144, 32)

In [7]:
xqrs.head()

,Patient,SWT_vs_XQRS,Pan_vs_XQRS,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label,Key
193,8643,0.977827,0.795613,665.457589,1.942364,1.532157,0.0,0.0,0.0,0.000000,1.532157,664.06250,3.90625,0.002302,0.002919,90.164234,90.352941,89.824561,0.253177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
194,8643,0.977827,0.795613,672.135417,11.762073,11.504201,0.0,0.0,1.0,6.666667,11.718750,675.78125,46.87500,0.017435,0.017500,89.294479,94.814815,88.275862,1.582749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
195,8643,0.977827,0.795613,657.031250,10.460751,9.002380,0.0,0.0,1.0,6.666667,9.511195,660.15625,42.96875,0.014476,0.015921,91.342484,96.603774,90.352941,1.471285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
196,8643,0.977827,0.795613,660.677083,2.499767,1.786586,0.0,0.0,0.0,0.000000,1.808242,660.15625,7.81250,0.002737,0.003784,90.817138,91.428571,90.352941,0.332091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
197,8643,0.977827,0.795613,663.330078,6.723404,2.240189,0.0,0.0,0.0,0.000000,2.470529,664.06250,15.62500,0.003724,0.010136,90.461400,91.428571,89.302326,0.885948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005


In [8]:
len(xqrs.Key.unique())

3870

In [9]:
xqrs.drop(['SWT_vs_XQRS', 'Pan_vs_XQRS'], axis=1, inplace=True)
xqrs.head()

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label,Key
193,8643,665.457589,1.942364,1.532157,0.0,0.0,0.0,0.000000,1.532157,664.06250,3.90625,0.002302,0.002919,90.164234,90.352941,89.824561,0.253177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
194,8643,672.135417,11.762073,11.504201,0.0,0.0,1.0,6.666667,11.718750,675.78125,46.87500,0.017435,0.017500,89.294479,94.814815,88.275862,1.582749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
195,8643,657.031250,10.460751,9.002380,0.0,0.0,1.0,6.666667,9.511195,660.15625,42.96875,0.014476,0.015921,91.342484,96.603774,90.352941,1.471285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
196,8643,660.677083,2.499767,1.786586,0.0,0.0,0.0,0.000000,1.808242,660.15625,7.81250,0.002737,0.003784,90.817138,91.428571,90.352941,0.332091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005
197,8643,663.330078,6.723404,2.240189,0.0,0.0,0.0,0.000000,2.470529,664.06250,15.62500,0.003724,0.010136,90.461400,91.428571,89.302326,0.885948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,train_01_tcp_ar_8643_s002_2012_02_22_t005


# Exams

In [10]:
cols_to_keep = ['Key', 'Patient', 'hf', 'lf_hf_ratio', 'vlf', 'max_hr', 'lf', 'cvi', 'sd1', 'min_hr', 'sd2', 'sampen', 'median_nni', 'mean_hr', 'mean_nni', 'csi', 'Modified_csi', 'label']

In [11]:
xqrs = xqrs[cols_to_keep]
xqrs.head()

,Key,Patient,hf,lf_hf_ratio,vlf,max_hr,lf,cvi,sd1,min_hr,sd2,sampen,median_nni,mean_hr,mean_nni,csi,Modified_csi,label
193,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,90.352941,NaN,NaN,NaN,89.824561,NaN,NaN,664.06250,90.164234,665.457589,NaN,NaN,0.0
194,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,94.814815,NaN,NaN,NaN,88.275862,NaN,NaN,675.78125,89.294479,672.135417,NaN,NaN,0.0
195,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,96.603774,NaN,NaN,NaN,90.352941,NaN,NaN,660.15625,91.342484,657.031250,NaN,NaN,0.0
196,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,91.428571,NaN,NaN,NaN,90.352941,NaN,NaN,660.15625,90.817138,660.677083,NaN,NaN,0.0
197,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,91.428571,NaN,NaN,NaN,89.302326,NaN,NaN,664.06250,90.461400,663.330078,NaN,NaN,0.0


In [12]:
params = {}
for cols in xqrs.columns.tolist()[2:]:
    params[cols] = [lambda x: x.isnull().sum(), 
                    lambda x: x.count() + x.isnull().sum(), 
                    lambda x: (x.isnull().sum() / (x.count() + x.isnull().sum())) * 100]

In [13]:
count_nan_by_key = xqrs[['Key'] + xqrs.columns.tolist()[2:]]\
                                              .groupby('Key')\
                                              .agg(params)\
                                              .rename(columns={'<lambda_0>': 'NaN',
                                                               '<lambda_1>': 'total',
                                                               '<lambda_2>': '%'})

In [14]:
total = np.zeros(count_nan_by_key.shape[0],)
i = 0
for cols in xqrs.columns.tolist()[2:]:
    sub_total = []
    for index, row in count_nan_by_key[(cols, '%')].iteritems():
        sub_total.append(row)
    i += 1
    total = np.add(total, sub_total)


total = total / 28

In [15]:
count_nan_by_key[('Total', 'Percent')] = total

In [16]:
crisis = xqrs[['Key', 'label']].groupby('Key').agg(lambda x: (x != 0).sum())

In [17]:
count_nan_by_key[('Total', 'Crisis')] = crisis.label.values

In [18]:
count_nan_by_key

hf                    \
                                              NaN total           %   
Key                                                                   
dev_01_tcp_ar_10022_s001_2013_02_10_t000      3.0   3.0  100.000000   
dev_01_tcp_ar_10022_s002_2013_02_10_t000     17.0  61.0   27.868852   
dev_01_tcp_ar_10022_s002_2013_02_10_t001     16.0  61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t002     16.0  61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t003     16.0  61.0   26.229508   
...                                           ...   ...         ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006  16.0  61.0   26.229508   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007  16.0  30.0   53.333333   
train_03_tcp_ar_a_9162_s002_2012_04_09_t008  17.0  61.0   27.868852   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000  26.0  26.0  100.000000   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001  51.0  61.0   83.606557   

                                            lf_hf_ratio                    \
                                                    NaN total           %   
Key                                                                         
dev_01_tcp_ar_10022_s001_2013_02_10_t000            3.0   3.0  100.000000   
dev_01_tcp_ar_10022_s002_2013_02_10_t000           17.0  61.0   27.868852   
dev_01_tcp_ar_10022_s002_2013_02_10_t001           16.0  61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t002           16.0  61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t003           16.0  61.0   26.229508   
...                                                 ...   ...         ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006        16.0  61.0   26.229508   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007        16.0  30.0   53.333333   
train_03_tcp_ar_a_9162_s002_2012_04_09_t008        17.0  61.0   27.868852   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000        26.0  26.0  100.000000   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001        61.0  61.0  100.000000   

                                              vlf                   max_hr  \
                                              NaN total           %    NaN   
Key                                                                          
dev_01_tcp_ar_10022_s001_2013_02_10_t000      3.0   3.0  100.000000    0.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t000     17.0  61.0   27.868852    1.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t001     16.0  61.0   26.229508    0.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t002     16.0  61.0   26.229508    0.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t003     16.0  61.0   26.229508    0.0   
...                                           ...   ...         ...    ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006  16.0  61.0   26.229508    0.0   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007  16.0  30.0   53.333333    0.0   
train_03_tcp_ar_a_9162_s002_2012_04_09_t008  17.0  61.0   27.868852    0.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000  26.0  26.0  100.000000    1.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001  51.0  61.0   83.606557   50.0   

                                                                lf        \
                                            total          %   NaN total   
Key                                                                        
dev_01_tcp_ar_10022_s001_2013_02_10_t000      3.0   0.000000   3.0   3.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t000     61.0   1.639344  17.0  61.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t001     61.0   0.000000  16.0  61.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t002     61.0   0.000000  16.0  61.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t003     61.0   0.000000  16.0  61.0   
...                                           ...        ...   ...   ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006  61.0   0.000000  16.0  61.0   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007  30.0   0.000000  16.0  30.0   
train_03_tcp_ar_a_9162_s002_2012_04_09_t008 

In [19]:
count_nan_by_key[count_nan_by_key[('hf', 'total')] > 17]

hf                     \
                                              NaN  total           %   
Key                                                                    
dev_01_tcp_ar_10022_s002_2013_02_10_t000     17.0   61.0   27.868852   
dev_01_tcp_ar_10022_s002_2013_02_10_t001     16.0   61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t002     16.0   61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t003     16.0   61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t004     16.0  125.0   12.800000   
...                                           ...    ...         ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006  16.0   61.0   26.229508   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007  16.0   30.0   53.333333   
train_03_tcp_ar_a_9162_s002_2012_04_09_t008  17.0   61.0   27.868852   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000  26.0   26.0  100.000000   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001  51.0   61.0   83.606557   

                                            lf_hf_ratio                     \
                                                    NaN  total           %   
Key                                                                          
dev_01_tcp_ar_10022_s002_2013_02_10_t000           17.0   61.0   27.868852   
dev_01_tcp_ar_10022_s002_2013_02_10_t001           16.0   61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t002           16.0   61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t003           16.0   61.0   26.229508   
dev_01_tcp_ar_10022_s002_2013_02_10_t004           16.0  125.0   12.800000   
...                                                 ...    ...         ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006        16.0   61.0   26.229508   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007        16.0   30.0   53.333333   
train_03_tcp_ar_a_9162_s002_2012_04_09_t008        17.0   61.0   27.868852   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000        26.0   26.0  100.000000   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001        61.0   61.0  100.000000   

                                              vlf                    max_hr  \
                                              NaN  total           %    NaN   
Key                                                                           
dev_01_tcp_ar_10022_s002_2013_02_10_t000     17.0   61.0   27.868852    1.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t001     16.0   61.0   26.229508    0.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t002     16.0   61.0   26.229508    0.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t003     16.0   61.0   26.229508    0.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t004     16.0  125.0   12.800000    0.0   
...                                           ...    ...         ...    ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006  16.0   61.0   26.229508    0.0   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007  16.0   30.0   53.333333    0.0   
train_03_tcp_ar_a_9162_s002_2012_04_09_t008  17.0   61.0   27.868852    0.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000  26.0   26.0  100.000000    1.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001  51.0   61.0   83.606557   50.0   

                                                                 lf         \
                                             total          %   NaN  total   
Key                                                                          
dev_01_tcp_ar_10022_s002_2013_02_10_t000      61.0   1.639344  17.0   61.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t001      61.0   0.000000  16.0   61.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t002      61.0   0.000000  16.0   61.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t003      61.0   0.000000  16.0   61.0   
dev_01_tcp_ar_10022_s002_2013_02_10_t004     125.0   0.000000  16.0  125.0   
...                                            ...        ...   ...    ...   
train_03_tcp_ar_a_9162_s002_2012_04_09_t006   61.0   0.000000  16.0   61.0   
train_03_tcp_ar_a_9162_s002_2012_04_09_t007   30.0   0.000

In [20]:
exams_to_keep = count_nan_by_key[count_nan_by_key[('hf', 'total')] > 17].index.tolist()

In [21]:
xqrs.shape

(238144, 18)

In [22]:
len(xqrs.Key.unique())

3870

In [23]:
xqrs = xqrs[xqrs.Key.isin(exams_to_keep)]
xqrs.shape

(235683, 18)

In [24]:
len(xqrs.Key.unique())

3575

In [25]:
xqrs.head()

,Key,Patient,hf,lf_hf_ratio,vlf,max_hr,lf,cvi,sd1,min_hr,sd2,sampen,median_nni,mean_hr,mean_nni,csi,Modified_csi,label
193,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,90.352941,NaN,NaN,NaN,89.824561,NaN,NaN,664.06250,90.164234,665.457589,NaN,NaN,0.0
194,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,94.814815,NaN,NaN,NaN,88.275862,NaN,NaN,675.78125,89.294479,672.135417,NaN,NaN,0.0
195,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,96.603774,NaN,NaN,NaN,90.352941,NaN,NaN,660.15625,91.342484,657.031250,NaN,NaN,0.0
196,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,91.428571,NaN,NaN,NaN,90.352941,NaN,NaN,660.15625,90.817138,660.677083,NaN,NaN,0.0
197,train_01_tcp_ar_8643_s002_2012_02_22_t005,8643,NaN,NaN,NaN,91.428571,NaN,NaN,NaN,89.302326,NaN,NaN,664.06250,90.461400,663.330078,NaN,NaN,0.0


In [26]:
params = {}
for cols in xqrs.columns.tolist()[2:]:
    params[cols] = [lambda x: x.isnull().sum(), 
                    lambda x: x.count() + x.isnull().sum(), 
                    lambda x: (x.isnull().sum() / (x.count() + x.isnull().sum())) * 100]

In [27]:
count_nan_by_key = xqrs[['Key'] + xqrs.columns.tolist()[2:]]\
                                              .groupby('Key')\
                                              .agg(params)\
                                              .rename(columns={'<lambda_0>': 'NaN',
                                                               '<lambda_1>': 'total',
                                                               '<lambda_2>': '%'})

In [28]:
total = np.zeros(count_nan_by_key.shape[0],)
i = 0
for cols in xqrs.columns.tolist()[2:]:
    sub_total = []
    for index, row in count_nan_by_key[(cols, '%')].iteritems():
        sub_total.append(row)
    i += 1
    total = np.add(total, sub_total)


total = total / 28

In [29]:
count_nan_by_key[('Total', 'Percent')] = total

In [30]:
crisis = xqrs[['Key', 'label']].groupby('Key').agg(lambda x: (x != 0).sum())

In [31]:
count_nan_by_key[('Total', 'Crisis')] = crisis.label.values

In [32]:
count_nan_by_key.columns[count_nan_by_key.columns.get_level_values(1) == 'total'].tolist()

[('hf', 'total'),
 ('lf_hf_ratio', 'total'),
 ('vlf', 'total'),
 ('max_hr', 'total'),
 ('lf', 'total'),
 ('cvi', 'total'),
 ('sd1', 'total'),
 ('min_hr', 'total'),
 ('sd2', 'total'),
 ('sampen', 'total'),
 ('median_nni', 'total'),
 ('mean_hr', 'total'),
 ('mean_nni', 'total'),
 ('csi', 'total'),
 ('Modified_csi', 'total'),
 ('label', 'total')]

In [33]:
count_nan_by_key[
 (count_nan_by_key[('hf', '%')] > 99) |\
 (count_nan_by_key[('lf_hf_ratio', '%')] > 99) |\
 (count_nan_by_key[('vlf', '%')] > 99) |\
 (count_nan_by_key[('max_hr', '%')] > 99) |\
 (count_nan_by_key[('lf', '%')] > 99) |\
 (count_nan_by_key[('cvi', '%')] > 99) |\
 (count_nan_by_key[('sd1', '%')] > 99) |\
 (count_nan_by_key[('min_hr', '%')] > 99) |\
 (count_nan_by_key[('sd2', '%')] > 99) |\
 (count_nan_by_key[('sampen', '%')] > 99) |\
 (count_nan_by_key[('median_nni', '%')] > 99) |\
 (count_nan_by_key[('mean_hr', '%')] > 99) |\
 (count_nan_by_key[('mean_nni', '%')] > 99) |\
 (count_nan_by_key[('csi', '%')] > 99) |\
 (count_nan_by_key[('Modified_csi', '%')] > 99) |\
 (count_nan_by_key[('label', '%')] > 99)
]

_10_t001   98.360656   60.0   61.0   

                                                        min_hr         \
                                                      %    NaN  total   
Key                                                                     
dev_01_tcp_ar_9570_s002_2012_11_05_t011       50.000000    3.0   18.0   
dev_01_tcp_ar_9697_s006_2013_05_17_t008       35.714286    2.0   28.0   
train_01_tcp_ar_10843_s010_2014_10_08_t002    42.105263    0.0   19.0   
train_01_tcp_ar_11081_s003_2013_12_26_t002    53.333333    0.0   30.0   
train_01_tcp_ar_11454_s003_2014_02_10_t001    37.500000    0.0   24.0   
train_01_tcp_ar_11972_s006_2014_09_17_t001    79.120879   57.0   91.0   
train_01_tcp_ar_12966_s005_2015_09_04_t003    86.666667    3.0   30.0   
train_01_tcp_ar_13407_s003_2015_09_30_t006    71.428571    1.0   21.0   
train_01_tcp_ar_2806_s001_2004_12_23_t006     31.818182    0.0   22.0   
train_01_tcp_ar_3101_s004_2010_04_02_t000    100.000000  121.0  124.0   
train_01_tcp_ar_6440_s006_2010_04_19_t000    100.000000  113.0  115.0   
train_01_tcp_ar_6455_s007_2014_09_15_t003     30.434783    0.0   23.0   
train_01_tcp_ar_6698_s001_2010_06_11_t001     92.727273   43.0   55.0   
train_01_tcp_ar_8295_s008_2012_07_23_t006     45.161290    1.0   31.0   
train_01_tcp_ar_8444_s003_2012_03_18_t011     96.428571   15.0   28.0   
train_01_tcp_ar_8444_s004_2012_03_19_t007     46.666667    0.0   30.0   
train_01_tcp_ar_8481_s002_2012_09_13_t003     54.545455    0.0   22.0   
train_01_tcp_ar_8481_s003_2012_09_13_t004     41.379310    2.0   29.0   
train_01_tcp_ar_9623_s006_2012_10_21_t007     46.666667    0.0   30.0   
train_01_tcp_ar_9852_s003_2013_09_25_t003     68.518519   23.0   54.0   
train_02_tcp_le_526_s002_2003_08_11_t006      38.095238    1.0   21.0   
train_03_tcp_ar_a_6514_s004_2010_04_28_t001   71.428571   14.0   21.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000   30.769231    1.0   26.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001   98.360656   50.0   61.0   

                                                          sd2         \
                                                     %    NaN  total   
Key                                                                    
dev_01_tcp_ar_9570_s002_2012_11_05_t011      16.666667    9.0   18.0   
dev_01_tcp_ar_9697_s006_2013_05_17_t008       7.142857   10.0   28.0   
train_01_tcp_ar_10843_s010_2014_10_08_t002    0.000000    8.0   19.0   
train_01_tcp_ar_11081_s003_2013_12_26_t002    0.000000   16.0   30.0   
train_01_tcp_ar_11454_s003_2014_02_10_t001    0.000000    9.0   24.0   
train_01_tcp_ar_11972_s006_2014_09_17_t001   62.637363   72.0   91.0   
train_01_tcp_ar_12966_s005_2015_09_04_t003   10.000000   26.0   30.0   
train_01_tcp_ar_13407_s003_2015_09_30_t006    4.761905   15.0   21.0   
train_01_tcp_ar_2806_s001_2004_12_23_t006     0.000000    7.0   22.0   
train_01_tcp_ar_3101_s004_2010_04_02_t000    97.580645  124.0  124.0   
train_01_tcp_ar_6440_s006_2010_04_19_t000    98.260870  115.0  115.0   
train_01_tcp_ar_6455_s007_2014_09_15_t003     0.000000    7.0   23.0   
train_01_tcp_ar_6698_s001_2010_06_11_t001    78.181818   51.0   55.0   
train_01_tcp_ar_8295_s008_2012_07_23_t006     3.225806   14.0   31.0   
train_01_tcp_ar_8444_s003_2012_03_18_t011    53.571429   27.0   28.0   
train_01_tcp_ar_8444_s004_2012_03_19_t007     0.000000   14.0   30.0   
train_01_tcp_ar_8481_s002_2012_09_13_t003     0.000000   12.0   22.0   
train_01_tcp_ar_8481_s003_2012_09_13_t004     6.896552   12.0   29.0   
train_01_tcp_ar_9623_s006_2012_10_21_t007     0.000000   14.0   30.0   
train_01_tcp_ar_9852_s003_2013_09_25_t003    42.592593   37.0   54.0   
train_02_tcp_le_526_s002_2003_08_11_t006      4.761905    8.0   21.0   
train_03_tcp_ar_a_6514_s004_2010_04_28_t001  66.666667   15.0   21.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t000   3.846154    8.0   26.0   
train_03_tcp_ar_a_9162_s003_2012_04_10_t001  81.967213   60.0   61.0   

                                            

In [34]:
exams_to_delete = count_nan_by_key[
 (count_nan_by_key[('hf', '%')] > 99) |\
 (count_nan_by_key[('lf_hf_ratio', '%')] > 99) |\
 (count_nan_by_key[('vlf', '%')] > 99) |\
 (count_nan_by_key[('max_hr', '%')] > 99) |\
 (count_nan_by_key[('lf', '%')] > 99) |\
 (count_nan_by_key[('cvi', '%')] > 99) |\
 (count_nan_by_key[('sd1', '%')] > 99) |\
 (count_nan_by_key[('min_hr', '%')] > 99) |\
 (count_nan_by_key[('sd2', '%')] > 99) |\
 (count_nan_by_key[('sampen', '%')] > 99) |\
 (count_nan_by_key[('median_nni', '%')] > 99) |\
 (count_nan_by_key[('mean_hr', '%')] > 99) |\
 (count_nan_by_key[('mean_nni', '%')] > 99) |\
 (count_nan_by_key[('csi', '%')] > 99) |\
 (count_nan_by_key[('Modified_csi', '%')] > 99) |\
 (count_nan_by_key[('label', '%')] > 99)
].index.tolist()

In [35]:
xqrs.shape

(235683, 18)

In [36]:
len(xqrs.Key.unique())

3575

In [37]:
xqrs = xqrs[~xqrs.Key.isin(exams_to_delete)]
xqrs.shape

(234730, 18)

In [38]:
len(xqrs.Key.unique())

3551

# Optionnal step

In [42]:
exams_to_delete = count_nan_by_key[
 (abs((count_nan_by_key[('hf', 'total')]) - (count_nan_by_key[('Total', 'Crisis')])) < 5) |\
 (abs((count_nan_by_key[('hf', 'total')]) - (count_nan_by_key[('Total', 'Crisis')])) > (count_nan_by_key[('hf', 'total')]) - 5)
].index.tolist()

In [43]:
xqrs = xqrs[~xqrs.Key.isin(exams_to_delete)]
xqrs.shape

(44927, 18)

In [44]:
len(xqrs.Key.unique())

504

# Patients

In [45]:
xqrs.head()

,Key,Patient,hf,lf_hf_ratio,vlf,max_hr,lf,cvi,sd1,min_hr,sd2,sampen,median_nni,mean_hr,mean_nni,csi,Modified_csi,label
2321,train_01_tcp_ar_630_s002_2003_05_28_t001,630,NaN,NaN,NaN,75.949367,NaN,NaN,NaN,71.856287,NaN,NaN,830.0,72.764862,824.791667,NaN,NaN,0.00000
2322,train_01_tcp_ar_630_s002_2003_05_28_t001,630,NaN,NaN,NaN,121.212121,NaN,NaN,NaN,89.552239,NaN,NaN,535.0,107.856219,561.470588,NaN,NaN,0.99025
2323,train_01_tcp_ar_630_s002_2003_05_28_t001,630,NaN,NaN,NaN,130.434783,NaN,NaN,NaN,111.627907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000
2324,train_01_tcp_ar_630_s002_2003_05_28_t001,630,NaN,NaN,NaN,187.500000,NaN,NaN,NaN,112.676056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000
2325,train_01_tcp_ar_630_s002_2003_05_28_t001,630,NaN,NaN,NaN,193.548387,NaN,NaN,NaN,122.020972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000


In [46]:
params = {}
for cols in xqrs.columns.tolist()[2:]:
    params[cols] = [lambda x: x.isnull().sum(), 
                    lambda x: x.count() + x.isnull().sum(), 
                    lambda x: (x.isnull().sum() / (x.count() + x.isnull().sum())) * 100]

In [47]:
count_nan_by_key = xqrs[['Patient'] + xqrs.columns.tolist()[2:]]\
                                              .groupby('Patient')\
                                              .agg(params)\
                                              .rename(columns={'<lambda_0>': 'NaN',
                                                               '<lambda_1>': 'total',
                                                               '<lambda_2>': '%'})

In [48]:
count_nan_by_key.shape

(151, 48)

In [49]:
total = np.zeros(count_nan_by_key.shape[0],)
i = 0
for cols in xqrs.columns.tolist()[2:]:
    sub_total = []
    for index, row in count_nan_by_key[(cols, '%')].iteritems():
        sub_total.append(row)
    i += 1
    total = np.add(total, sub_total)


total = total / 28

In [50]:
count_nan_by_key[('Total', 'Percent')] = total

In [51]:
count_nan_by_key[count_nan_by_key[('Total', 'Percent')] < 2].sort_values(('Total', 'Percent'))

hf                  lf_hf_ratio                    vlf         \
          NaN  total         %         NaN  total         %   NaN  total   
Patient                                                                    
5943     17.0  280.0  6.071429        17.0  280.0  6.071429  17.0  280.0   
9842     32.0  498.0  6.425703        32.0  498.0  6.425703  32.0  498.0   
10489    16.0  232.0  6.896552        16.0  232.0  6.896552  16.0  232.0   
7252     16.0  199.0  8.040201        16.0  199.0  8.040201  16.0  199.0   
4671     16.0  193.0  8.290155        16.0  193.0  8.290155  16.0  193.0   

                  max_hr                lf                    cvi         \
                %    NaN  total    %   NaN  total         %   NaN  total   
Patient                                                                    
5943     6.071429    0.0  280.0  0.0  17.0  280.0  6.071429   8.0  280.0   
9842     6.425703    0.0  498.0  0.0  32.0  498.0  6.425703  14.0  498.0   
10489    6.896552    0.0  232.0  0.0  16.0  232.0  6.896552   7.0  232.0   
7252     8.040201    0.0  199.0  0.0  16.0  199.0  8.040201   7.0  199.0   
4671     8.290155    0.0  193.0  0.0  16.0  193.0  8.290155   7.0  193.0   

                    sd1                  min_hr               sd2         \
                %   NaN  total         %    NaN  total    %   NaN  total   
Patient                                                                    
5943     2.857143   8.0  280.0  2.857143    0.0  280.0  0.0   8.0  280.0   
9842     2.811245  14.0  498.0  2.811245    0.0  498.0  0.0  14.0  498.0   
10489    3.017241   7.0  232.0  3.017241    0.0  232.0  0.0   7.0  232.0   
7252     3.517588   7.0  199.0  3.517588    0.0  199.0  0.0   7.0  199.0   
4671     3.626943   7.0  193.0  3.626943    0.0  193.0  0.0   7.0  193.0   

                  sampen                  median_nni                  mean_hr  \
                %    NaN  total         %        NaN  total         %     NaN   
Patient                                                                         
5943     2.857143    7.0  280.0  2.500000        1.0  280.0  0.357143     1.0   
9842     2.811245   14.0  498.0  2.811245        0.0  498.0  0.000000     0.0   
10489    3.017241    7.0  232.0  3.017241        0.0  232.0  0.000000     0.0   
7252     3.517588    7.0  199.0  3.517588        0.0  199.0  0.000000     0.0   
4671     3.626943    7.0  193.0  3.626943        0.0  193.0  0.000000     0.0   

                         mean_nni                    csi                   \
         total         %      NaN  total         %   NaN  total         %   
Patient                                                                     
5943     280.0  0.357143      1.0  280.0  0.357143   8.0  280.0  2.857143   
9842     498.0  0.000000      0.0  498.0  0.000000  14.0  498.0  2.811245   
10489    232.0  0.000000      0.0  232.0  0.000000   7.0  232.0  3.017241   
7252     199.0  0.000000      0.0  199.0  0.000000   7.0  199.0  3.517588   
4671     193.0  0.000000      0.0  193.0  0.000000   7.0  193.0  3.626943   

        Modified_csi                  label                      Total  
                 NaN  total         %   NaN  total         %   Percent  
Patient                                                                 
5943             8.0  280.0  2.857143   0.0  280.0  0.000000  1.505102  
9842            14.0  498.0  2.811245   2.0  498.0  0.401606  1.534710  
10489            7.0  232.0  3.017241   1.0  232.0  0.431034  1.647167  
7252             7.0  199.0  3.517588   1.0  199.0  0.502513  1.920316  
4671             7.0  193.0  3.626943   0.0  193.0  0.000000  1.961510

In [52]:
crisis = xqrs[['Patient', 'label']].groupby('Patient').agg(lambda x: (x != 0).sum())

In [53]:
count_nan_by_key[('Total', 'Crisis')] = crisis.label.values

In [54]:
count_nan_by_key[(count_nan_by_key[('Total', 'Percent')] < 3) & (count_nan_by_key[('Total', 'Crisis')] > 1)].sort_values([('Total', 'Crisis')], ascending=False)

hf                   lf_hf_ratio                     vlf         \
          NaN  total          %         NaN  total          %   NaN  total   
Patient                                                                      
10489    16.0  232.0   6.896552        16.0  232.0   6.896552  16.0  232.0   
5943     17.0  280.0   6.071429        17.0  280.0   6.071429  17.0  280.0   
7252     16.0  199.0   8.040201        16.0  199.0   8.040201  16.0  199.0   
258      32.0  305.0  10.491803        32.0  305.0  10.491803  32.0  305.0   
5034     21.0  213.0   9.859155        21.0  213.0   9.859155  21.0  213.0   
9842     32.0  498.0   6.425703        32.0  498.0   6.425703  32.0  498.0   
6811     34.0  353.0   9.631728        34.0  353.0   9.631728  34.0  353.0   
4671     16.0  193.0   8.290155        16.0  193.0   8.290155  16.0  193.0   
7793     16.0  138.0  11.594203        16.0  138.0  11.594203  16.0  138.0   
9570     16.0  163.0   9.815951        16.0  163.0   9.815951  16.0  163.0   
8480     33.0  318.0  10.377358        33.0  318.0  10.377358  33.0  318.0   
1113     32.0  282.0  11.347518        32.0  282.0  11.347518  32.0  282.0   
1324     16.0  129.0  12.403101        16.0  129.0  12.403101  16.0  129.0   
8295     16.0  167.0   9.580838        16.0  167.0   9.580838  16.0  167.0   
529      32.0  271.0  11.808118        32.0  271.0  11.808118  32.0  271.0   
7835     17.0  151.0  11.258278        17.0  151.0  11.258278  17.0  151.0   
9994     32.0  270.0  11.851852        32.0  270.0  11.851852  32.0  270.0   
3635     16.0  162.0   9.876543        16.0  162.0   9.876543  16.0  162.0   
1981     33.0  278.0  11.870504        33.0  278.0  11.870504  33.0  278.0   
6139     16.0  131.0  12.213740        16.0  131.0  12.213740  16.0  131.0   
3306     16.0  143.0  11.188811        16.0  143.0  11.188811  16.0  143.0   
7458     17.0  156.0  10.897436        17.0  156.0  10.897436  17.0  156.0   
4126     16.0  131.0  12.213740        16.0  131.0  12.213740  16.0  131.0   
10427    31.0  262.0  11.832061        31.0  262.0  11.832061  31.0  262.0   

                   max_hr                     lf                     cvi  \
                 %    NaN  total         %   NaN  total          %   NaN   
Patient                                                                    
10489     6.896552    0.0  232.0  0.000000  16.0  232.0   6.896552   7.0   
5943      6.071429    0.0  280.0  0.000000  17.0  280.0   6.071429   8.0   
7252      8.040201    0.0  199.0  0.000000  16.0  199.0   8.040201   7.0   
258      10.491803    1.0  305.0  0.327869  32.0  305.0  10.491803  14.0   
5034      9.859155    0.0  213.0  0.000000  21.0  213.0   9.859155  12.0   
9842      6.425703    0.0  498.0  0.000000  32.0  498.0   6.425703  14.0   
6811      9.631728    0.0  353.0  0.000000  34.0  353.0   9.631728  16.0   
4671      8.290155    0.0  193.0  0.000000  16.0  193.0   8.290155   7.0   
7793     11.594203    1.0  138.0  0.724638  16.0  138.0  11.594203   7.0   
9570      9.815951    0.0  163.0  0.000000  16.0  163.0   9.815951   7.0   
8480     10.377358    0.0  318.0  0.000000  33.0  318.0  10.377358  15.0   
1113     11.347518    0.0  282.0  0.000000  32.0  282.0  11.347518  14.0   
1324     12.403101    0.0  129.0  0.000000  16.0  129.0  12.403101   7.0   
8295      9.580838    0.0  167.0  0.000000  16.0  167.0   9.580838   7.0   
529      11.808118    0.0  271.0  0.000000  32.0  271.0  11.808118  14.0   
7835     11.258278    0.0  151.0  0.000000  17.0  151.0  11.258278   8.0   
9994     11.851852    0.0  270.0  0.000000  32.0  270.0  11.851852  14.0   
3635      9.876543    0.0  162.0  0.000000  16.0  162.0   9.876543   7.0   
1981     11.870504    0.0  278.0  0.000000  33.0  278.0  11.870504  15.0   
6139     12.213740    0.0  131.0  0.000000  16.0  131.0  12.213740   7.0   
3306     11.188811    1.0  143.0  0.699301  16.0  143.0  11.188811   7.0   
7458     10.897436    0.0  156.0  0.000000  17.0  156.0  10.897436   8.0   
4126

In [55]:
count_nan_by_key.sort_values([('Total', 'Crisis')], ascending=False)

hf                    lf_hf_ratio                       vlf  \
           NaN   total          %         NaN   total          %    NaN   
Patient                                                                   
9578     322.0  2521.0  12.772709       322.0  2521.0  12.772709  322.0   
8544     410.0  1643.0  24.954352       410.0  1643.0  24.954352  410.0   
10418    361.0  1307.0  27.620505       361.0  1307.0  27.620505  361.0   
11077    368.0  1711.0  21.507890       368.0  1711.0  21.507890  368.0   
9104     401.0  2013.0  19.920517       401.0  2013.0  19.920517  401.0   
...        ...     ...        ...         ...     ...        ...    ...   
3546      46.0   121.0  38.016529        46.0   121.0  38.016529   46.0   
6059      20.0   130.0  15.384615        20.0   130.0  15.384615   20.0   
775       46.0    88.0  52.272727        59.0    88.0  67.045455   46.0   
6455      16.0    22.0  72.727273        16.0    22.0  72.727273   16.0   
5346      16.0    35.0  45.714286        16.0    35.0  45.714286   16.0   

                           max_hr                        lf          \
          total          %    NaN   total          %    NaN   total   
Patient                                                               
9578     2521.0  12.772709    4.0  2521.0   0.158667  322.0  2521.0   
8544     1643.0  24.954352    5.0  1643.0   0.304321  410.0  1643.0   
10418    1307.0  27.620505    4.0  1307.0   0.306044  361.0  1307.0   
11077    1711.0  21.507890    3.0  1711.0   0.175336  368.0  1711.0   
9104     2013.0  19.920517    3.0  2013.0   0.149031  401.0  2013.0   
...         ...        ...    ...     ...        ...    ...     ...   
3546      121.0  38.016529    0.0   121.0   0.000000   46.0   121.0   
6059      130.0  15.384615    0.0   130.0   0.000000   20.0   130.0   
775        88.0  52.272727   29.0    88.0  32.954545   46.0    88.0   
6455       22.0  72.727273    0.0    22.0   0.000000   16.0    22.0   
5346       35.0  45.714286    0.0    35.0   0.000000   16.0    35.0   

                      cvi                       sd1                    min_hr  \
                 %    NaN   total          %    NaN   total          %    NaN   
Patient                                                                         
9578     12.772709  142.0  2521.0   5.632685  142.0  2521.0   5.632685    4.0   
8544     24.954352  186.0  1643.0  11.320755  186.0  1643.0  11.320755    5.0   
10418    27.620505  187.0  1307.0  14.307575  187.0  1307.0  14.307575    4.0   
11077    21.507890  162.0  1711.0   9.468147  162.0  1711.0   9.468147    3.0   
9104     19.920517  177.0  2013.0   8.792846  177.0  2013.0   8.792846    3.0   
...            ...    ...     ...        ...    ...     ...        ...    ...   
3546     38.016529   42.0   121.0  34.710744   42.0   121.0  34.710744    0.0   
6059     15.384615   11.0   130.0   8.461538   11.0   130.0   8.461538    0.0   
775      52.272727   37.0    88.0  42.045455   37.0    88.0  42.045455   29.0   
6455     72.727273    9.0    22.0  40.909091    9.0    22.0  40.909091    0.0   
5346     45.714286    7.0    35.0  20.000000    7.0    35.0  20.000000    0.0   

                              sd2                    sampen          \
          total          %    NaN   total          %    NaN   total   
Patient                                                               
9578     2521.0   0.158667  142.0  2521.0   5.632685  126.0  2521.0   
8544     1643.0   0.304321  186.0  1643.0  11.320755  168.0  1643.0   
10418    1307.0   0.306044  187.0  1307.0  14.307575  126.0  1307.0   
11077    1711.0   0.175336  162.0  1711.0   9.468147  161.0  1711.0   
9104     2013.0   0.149031  177.0  2013.0   8.792846  175.0  2013.0   
...         ...        ...    ...     ...        ...    ...     ...   
3546      121.0   0.000000   42.0   121.0  34.710744    7.0   121.0   
6059      130.0   0.000000   11.0   130.0   8.461538    7.0   130.0   
775        88.0  32.954545   37.0    88.0  42.0454

In [56]:
count_nan_by_key[(count_nan_by_key[('Total', 'Percent')] < 9)\
 & ((count_nan_by_key[('Total', 'Crisis')] / count_nan_by_key[('hf', 'total')]) * 100 > 10)\
  & (abs(count_nan_by_key[('Total', 'Crisis')] - count_nan_by_key[('hf', 'total')]) > 2)].sort_values([('Total', 'Crisis')], ascending=False)

2760     0.000000    7.0   121.0   5.785124    7.0   121.0   5.785124   
10760    0.000000    7.0   104.0   6.730769    7.0   104.0   6.730769   
5672     0.000000    7.0   127.0   5.511811    7.0   127.0   5.511811   
574      0.000000   10.0   121.0   8.264463    7.0   121.0   5.785124   
7835     0.000000    8.0   151.0   5.298013    7.0   151.0   4.635762   
2484     0.000000    7.0   124.0   5.645161    7.0   124.0   5.645161   
9232     0.000000    7.0    58.0  12.068966    7.0    58.0  12.068966   
3884     0.000000    7.0   116.0   6.034483    7.0   116.0   6.034483   
11321    0.000000    7.0    61.0  11.475410    7.0    61.0  11.475410   
5427     0.000000    8.0    61.0  13.114754    7.0    61.0  11.475410   
8092     0.000000    7.0    44.0  15.909091    7.0    44.0  15.909091   
2445     0.000000   10.0    67.0  14.925373    7.0    67.0  10.447761   
11272    0.000000    8.0    61.0  13.114754    7.0    61.0  11.475410   

        median_nni                   mean_hr                   mean_nni  \
               NaN   total         %     NaN   total         %      NaN   
Patient                                                                   
9578          10.0  2521.0  0.396668    10.0  2521.0  0.396668     10.0   
8544          20.0  1643.0  1.217285    20.0  1643.0  1.217285     20.0   
10418         63.0  1307.0  4.820199    63.0  1307.0  4.820199     63.0   
11077          4.0  1711.0  0.233781     4.0  1711.0  0.233781      4.0   
9104           5.0  2013.0  0.248385     5.0  2013.0  0.248385      5.0   
11870         31.0  1331.0  2.329076    31.0  1331.0  2.329076     31.0   
302           14.0   503.0  2.783300    14.0   503.0  2.783300     14.0   
6440           3.0   704.0  0.426136     3.0   704.0  0.426136      3.0   
10489          0.0   232.0  0.000000     0.0   232.0  0.000000      0.0   
6904           0.0   309.0  0.000000     0.0   309.0  0.000000      0.0   
6175           0.0   335.0  0.000000     0.0   335.0  0.000000      0.0   
9839           9.0  1634.0  0.550796     9.0  1634.0  0.550796      9.0   
7128          24.0   723.0  3.319502    24.0   723.0  3.319502     24.0   
10591         17.0   407.0  4.176904    17.0   407.0  4.176904     17.0   
8460           4.0   283.0  1.413428     4.0   283.0  1.413428      4.0   
11869          1.0   909.0  0.110011     1.0   909.0  0.110011      1.0   
5943           1.0   280.0  0.357143     1.0   280.0  0.357143      1.0   
9231           5.0   194.0  2.577320     5.0   194.0  2.577320      5.0   
10106          2.0   838.0  0.238663     2.0   838.0  0.238663      2.0   
6514           2.0   198.0  1.010101     2.0   198.0  1.010101      2.0   
7252           0.0   199.0  0.000000     0.0   199.0  0.000000      0.0   
8889           8.0   855.0  0.935673     8.0   855.0  0.935673      8.0   
8476           3.0   432.0  0.694444     3.0   432.0  0.694444      3.0   
258            1.0   305.0  0.327869     1.0   305.0  0.327869      1.0   
5034           5.0   213.0  2.347418     5.0   213.0  2.347418      5.0   
12973         13.0   509.0  2.554028    13.0   509.0  2.554028     13.0   
6413           9.0   250.0  3.600000     9.0   250.0  3.600000      9.0   
12941          4.0   193.0  2.072539     4.0   193.0  2.072539      4.0   
6083           0.0   230.0  0.000000     0.0   230.0  0.000000      0.0   
9842           0.0   498.0  0.000000     0.0   498.0  0.000000      0.0   
7584          20.0   684.0  2.923977    20.0   684.0  2.923977     20.0   
2991           3.0   393.0  0.763359     3.0   393.0  0.763359      3.0   
7234           0.0   183.0  0.000000     0.0   183.0  0.000000      0.0   
3977           5.0   140.0  3.571429     5.0   140.0  3.571429      5.0   
5452           5.0   472.0  1.059322     5.0   472.0  1.059322      5.0   
7032           8.0   279.0  2.867384     8.0   279.0  2.867384      8.0   
2806           3.0   194.0  1.546392     3.0   194.0  1.546392      3.0   
10158          9.0   411.0  2.189781     9.0   411

In [57]:
count_nan_by_key[('Total', '%% of crisis')] = (count_nan_by_key[('Total', 'Crisis')] / count_nan_by_key[('label', 'total')]) * 100

In [58]:
count_nan_by_key

hf                   lf_hf_ratio                      vlf         \
           NaN  total          %         NaN  total          %    NaN  total   
Patient                                                                        
10022     44.0  181.0  24.309392        44.0  181.0  24.309392   44.0  181.0   
1006      51.0  201.0  25.373134        51.0  201.0  25.373134   51.0  201.0   
10062     32.0  121.0  26.446281        32.0  121.0  26.446281   32.0  121.0   
10106    128.0  838.0  15.274463       128.0  838.0  15.274463  128.0  838.0   
10158     82.0  411.0  19.951338        82.0  411.0  19.951338   82.0  411.0   
...        ...    ...        ...         ...    ...        ...    ...    ...   
9842      32.0  498.0   6.425703        32.0  498.0   6.425703   32.0  498.0   
9885      36.0  151.0  23.841060        36.0  151.0  23.841060   36.0  151.0   
9932      16.0   30.0  53.333333        16.0   30.0  53.333333   16.0   30.0   
9934      16.0   61.0  26.229508        16.0   61.0  26.229508   16.0   61.0   
9994      32.0  270.0  11.851852        32.0  270.0  11.851852   32.0  270.0   

                   max_hr                      lf                     cvi  \
                 %    NaN  total         %    NaN  total          %   NaN   
Patient                                                                     
10022    24.309392    0.0  181.0  0.000000   44.0  181.0  24.309392  18.0   
1006     25.373134    0.0  201.0  0.000000   51.0  201.0  25.373134  28.0   
10062    26.446281    1.0  121.0  0.826446   32.0  121.0  26.446281  15.0   
10106    15.274463    0.0  838.0  0.000000  128.0  838.0  15.274463  57.0   
10158    19.951338    1.0  411.0  0.243309   82.0  411.0  19.951338  39.0   
...            ...    ...    ...       ...    ...    ...        ...   ...   
9842      6.425703    0.0  498.0  0.000000   32.0  498.0   6.425703  14.0   
9885     23.841060    0.0  151.0  0.000000   36.0  151.0  23.841060  18.0   
9932     53.333333    0.0   30.0  0.000000   16.0   30.0  53.333333   7.0   
9934     26.229508    1.0   61.0  1.639344   16.0   61.0  26.229508   7.0   
9994     11.851852    0.0  270.0  0.000000   32.0  270.0  11.851852  14.0   

                            sd1                   min_hr                   \
         total          %   NaN  total          %    NaN  total         %   
Patient                                                                     
10022    181.0   9.944751  18.0  181.0   9.944751    0.0  181.0  0.000000   
1006     201.0  13.930348  28.0  201.0  13.930348    0.0  201.0  0.000000   
10062    121.0  12.396694  15.0  121.0  12.396694    1.0  121.0  0.826446   
10106    838.0   6.801909  57.0  838.0   6.801909    0.0  838.0  0.000000   
10158    411.0   9.489051  39.0  411.0   9.489051    1.0  411.0  0.243309   
...        ...        ...   ...    ...        ...    ...    ...       ...   
9842     498.0   2.811245  14.0  498.0   2.811245    0.0  498.0  0.000000   
9885     151.0  11.920530  18.0  151.0  11.920530    0.0  151.0  0.000000   
9932      30.0  23.333333   7.0   30.0  23.333333    0.0   30.0  0.000000   
9934      61.0  11.475410   7.0   61.0  11.475410    1.0   61.0  1.639344   
9994     270.0   5.185185  14.0  270.0   5.185185    0.0  270.0  0.000000   

          sd2                   sampen                   median_nni         \
          NaN  total          %    NaN  total          %        NaN  total   
Patient                                                                      
10022    18.0  181.0   9.944751   14.0  181.0   7.734807        6.0  181.0   
1006     28.0  201.0  13.930348   14.0  201.0   6.965174        7.0  201.0   
10062    15.0  121.0  12.396694   14.0  121.0  11.570248        2.0  121.0   
10106    57.0  838.0   6.801909   56.0  838.0   6.682578        2.0  838.0   
10158    39.0  411.0   9.489051   28.0  411.0   6.812652        9.0  411.0   
...       ...    ...        ...    ...    ...        ...        ...    ...   
9842     14.0  498.0   2.811245

In [59]:
count_nan_by_key[(count_nan_by_key[('Total', 'Percent')] < 9)\
 & ((count_nan_by_key[('Total', 'Crisis')] / count_nan_by_key[('hf', 'total')]) * 100 > 10)\
  & (abs(count_nan_by_key[('Total', 'Crisis')] - count_nan_by_key[('hf', 'total')]) > 2)].sort_values([('Total', '%% of crisis')], ascending=False)

10489          0.0   232.0  0.000000     0.0   232.0  0.000000      0.0   
9231           5.0   194.0  2.577320     5.0   194.0  2.577320      5.0   
6514           2.0   198.0  1.010101     2.0   198.0  1.010101      2.0   
7252           0.0   199.0  0.000000     0.0   199.0  0.000000      0.0   
6904           0.0   309.0  0.000000     0.0   309.0  0.000000      0.0   
8460           4.0   283.0  1.413428     4.0   283.0  1.413428      4.0   
3977           5.0   140.0  3.571429     5.0   140.0  3.571429      5.0   
6175           0.0   335.0  0.000000     0.0   335.0  0.000000      0.0   
5034           5.0   213.0  2.347418     5.0   213.0  2.347418      5.0   
5943           1.0   280.0  0.357143     1.0   280.0  0.357143      1.0   
302           14.0   503.0  2.783300    14.0   503.0  2.783300     14.0   
12941          4.0   193.0  2.072539     4.0   193.0  2.072539      4.0   
7234           0.0   183.0  0.000000     0.0   183.0  0.000000      0.0   
9578          10.0  2521.0  0.396668    10.0  2521.0  0.396668     10.0   
10591         17.0   407.0  4.176904    17.0   407.0  4.176904     17.0   
6413           9.0   250.0  3.600000     9.0   250.0  3.600000      9.0   
6507           4.0   122.0  3.278689     4.0   122.0  3.278689      4.0   
258            1.0   305.0  0.327869     1.0   305.0  0.327869      1.0   
6083           0.0   230.0  0.000000     0.0   230.0  0.000000      0.0   
2806           3.0   194.0  1.546392     3.0   194.0  1.546392      3.0   
473            3.0   129.0  2.325581     3.0   129.0  2.325581      3.0   
6440           3.0   704.0  0.426136     3.0   704.0  0.426136      3.0   
9934           1.0    61.0  1.639344     1.0    61.0  1.639344      1.0   
8544          20.0  1643.0  1.217285    20.0  1643.0  1.217285     20.0   
10418         63.0  1307.0  4.820199    63.0  1307.0  4.820199     63.0   
9630          12.0   200.0  6.000000    12.0   200.0  6.000000     12.0   
5633           0.0   183.0  0.000000     0.0   183.0  0.000000      0.0   
13085          1.0   103.0  0.970874     1.0   103.0  0.970874      1.0   
3401           1.0    89.0  1.123596     1.0    89.0  1.123596      1.0   
10062          2.0   121.0  1.652893     2.0   121.0  1.652893      2.0   
8476           3.0   432.0  0.694444     3.0   432.0  0.694444      3.0   
7032           8.0   279.0  2.867384     8.0   279.0  2.867384      8.0   
4671           0.0   193.0  0.000000     0.0   193.0  0.000000      0.0   
13103          4.0   189.0  2.116402     4.0   189.0  2.116402      4.0   
7793           1.0   138.0  0.724638     1.0   138.0  0.724638      1.0   
10843          1.0   265.0  0.377358     1.0   265.0  0.377358      1.0   
7128          24.0   723.0  3.319502    24.0   723.0  3.319502     24.0   
9885           5.0   151.0  3.311258     5.0   151.0  3.311258      5.0   
12973         13.0   509.0  2.554028    13.0   509.0  2.554028     13.0   
11870         31.0  1331.0  2.329076    31.0  1331.0  2.329076     31.0   
11077          4.0  1711.0  0.233781     4.0  1711.0  0.233781      4.0   
4473           0.0   121.0  0.000000     0.0   121.0  0.000000      0.0   
5625           0.0   121.0  0.000000     0.0   121.0  0.000000      0.0   
4151           1.0   143.0  0.699301     1.0   143.0  0.699301      1.0   
9232           0.0    58.0  0.000000     0.0    58.0  0.000000      0.0   
2991           3.0   393.0  0.763359     3.0   393.0  0.763359      3.0   
9570           0.0   163.0  0.000000     0.0   163.0  0.000000      0.0   
3668           9.0   126.0  7.142857     9.0   126.0  7.142857      9.0   
9842           0.0   498.0  0.000000     0.0   498.0  0.000000      0.0   
10760          1.0   104.0  0.961538     1.0   104.0  0.961538      1.0   
8527           1.0   249.0  0.401606     1.0   249.0  0.401606      1.0   
8092           0.0    44.0  0.000000     0.0    44.0  0.000000      0.0   
9104           5.0  2013.0  0.248385     5.0  2013.0  0.248385      5.0   
11083          2.0   228.

In [60]:
len(count_nan_by_key[(count_nan_by_key[('Total', 'Percent')] < 9)\
 & ((count_nan_by_key[('Total', 'Crisis')] / count_nan_by_key[('hf', 'total')]) * 100 > 10)\
  & (abs(count_nan_by_key[('Total', 'Crisis')] - count_nan_by_key[('hf', 'total')]) > 2)].sort_values([('Total', 'Crisis')], ascending=False).index.tolist())

86

In [61]:
count_nan_by_key[(count_nan_by_key[('Total', 'Percent')] < 9)\
 & ((count_nan_by_key[('Total', 'Crisis')] / count_nan_by_key[('hf', 'total')]) * 100 > 10)\
  & (abs(count_nan_by_key[('Total', 'Crisis')] - count_nan_by_key[('hf', 'total')]) > 2)].sort_values([('Total', 'Crisis')], ascending=False).index.tolist()

['9578',
 '8544',
 '10418',
 '11077',
 '9104',
 '11870',
 '302',
 '6440',
 '10489',
 '6904',
 '6175',
 '9839',
 '7128',
 '10591',
 '8460',
 '11869',
 '5943',
 '9231',
 '10106',
 '6514',
 '7252',
 '8889',
 '8476',
 '258',
 '5034',
 '12973',
 '6413',
 '12941',
 '6083',
 '9842',
 '7584',
 '2991',
 '7234',
 '3977',
 '5452',
 '7032',
 '2806',
 '10158',
 '10843',
 '9630',
 '6811',
 '5633',
 '4671',
 '13103',
 '6507',
 '11580',
 '473',
 '8527',
 '9697',
 '11083',
 '8444',
 '675',
 '7793',
 '9885',
 '4456',
 '10062',
 '9570',
 '8480',
 '13085',
 '9623',
 '4151',
 '12759',
 '10022',
 '7235',
 '4473',
 '5625',
 '3401',
 '492',
 '3668',
 '7623',
 '1324',
 '9934',
 '8295',
 '2760',
 '10760',
 '5672',
 '574',
 '7835',
 '2484',
 '9232',
 '3884',
 '11321',
 '5427',
 '8092',
 '2445',
 '11272']

In [175]:
patients = ['9578', '11333', '8544', '10418', '11077', '7032', '6175', '4456', '11870', '302', '6440', '9866', '2521', '1770', '6904', '6514', '8460', '7128', '11869', '12941', '10106', '5943', '7252', '7234', '6546', '258', '12973', '5034', '9540', '9842', '6413', '3401', '6083', '2991', '13085', '7584', '5452', '3977', '2806', '10843', '9630', '5633', '13103', '6507', '473', '675', '7793', '9885', '4151', '12759', '5625', '6444', '3668', '2448', '7623', '1324', '2760', '2484', '7835']

In [176]:
len(xqrs[xqrs.Patient.isin(patients)].Key.unique())

759

In [177]:
len(xqrs[xqrs.Patient.isin(patients)].Patient.unique())

59

In [178]:
xqrs[xqrs.Patient.isin(patients)].shape

(51010, 18)

In [179]:
final_dataset = xqrs[xqrs.Patient.isin(patients)]

In [180]:
final_dataset.drop(['Patient'], axis=1, inplace=True)

In [181]:
final_dataset.head()

,Key,hf,lf_hf_ratio,vlf,max_hr,lf,cvi,sd1,min_hr,sd2,sampen,median_nni,mean_hr,mean_nni,csi,Modified_csi,label
4930,train_01_tcp_ar_7793_s001_2011_05_27_t001,NaN,NaN,NaN,127.118644,NaN,NaN,NaN,105.633803,NaN,NaN,506.0,118.490493,507.8,NaN,NaN,0.000
4931,train_01_tcp_ar_7793_s001_2011_05_27_t001,NaN,NaN,NaN,120.000000,NaN,NaN,NaN,118.110236,NaN,NaN,504.0,119.049198,504.0,NaN,NaN,0.016
4932,train_01_tcp_ar_7793_s001_2011_05_27_t001,NaN,NaN,NaN,120.967742,NaN,NaN,NaN,118.110236,NaN,NaN,502.0,119.532917,502.0,NaN,NaN,1.000
4933,train_01_tcp_ar_7793_s001_2011_05_27_t001,NaN,NaN,NaN,120.967742,NaN,NaN,NaN,116.279070,NaN,NaN,512.0,117.713266,509.8,NaN,NaN,1.000
4934,train_01_tcp_ar_7793_s001_2011_05_27_t001,NaN,NaN,NaN,120.967742,NaN,NaN,NaN,118.110236,NaN,NaN,504.0,118.816291,505.0,NaN,NaN,1.000


In [182]:
final_dataset.shape

(51010, 17)

In [183]:
final_dataset.to_csv('../to_csv/final_dataset/xqrs_filtered.csv', index=False)

# V2 dataset

In [62]:
patients_V2 = ['9578', '8544', '10418', '11077', '9104', '11870', '302', '6440', '10489', '6904', '6175', '9839', '7128', '10591', '8460', '11869', '5943', '9231', '10106', '6514', '7252', '8889', '8476', '258', '5034', '12973', '6413', '12941', '6083', '9842', '7584', '2991', '7234', '3977', '5452', '7032', '2806', '10158', '10843', '9630', '6811', '5633', '4671', '13103', '6507', '11580', '473', '8527', '9697', '11083', '8444', '675', '7793', '9885', '4456', '10062', '9570', '8480', '13085', '9623', '4151', '12759', '10022', '7235', '4473', '5625', '3401', '492', '3668', '7623', '1324', '9934', '8295', '2760', '10760', '5672', '574', '7835', '2484', '9232', '3884', '11321', '5427', '8092', '2445', '11272']

In [63]:
len(xqrs[xqrs.Patient.isin(patients_V2)].Key.unique())

376

In [64]:
xqrs[xqrs.Patient.isin(patients_V2)].shape

(32971, 18)

In [65]:
len(xqrs[xqrs.Patient.isin(patients_V2)].Patient.unique())

86

In [66]:
final_dataset_v2 = xqrs[xqrs.Patient.isin(patients_V2)]

In [67]:
final_dataset_v2.drop(['Patient'], axis=1, inplace=True)

In [68]:
final_dataset_v2.shape

(32971, 17)

In [69]:
final_dataset_v2.to_csv('../to_csv/final_dataset/xqrs_filtered_v2.csv', index=False)